# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
sales:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
temp3:
  0: NB(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

Converting Master1 to the Second or Third normal form is hard to do directly. A simpler approach consists in using the Boyce-Codd normal form algorithm. Then, because the Boyce-Codd normal form implies the second and third normal forms, we will be done (see slide 76 in course to justify that BCNF -> 3F).

The main functional dependencies in Master1 are the following:

{NV} $\rightarrow$ {NV, CRU, DEGRE, MILL} <br>
{NP} $\rightarrow$ {NP, NOM, PRENOM, REGION} <br>

The other dependencies are trivial dependencies, which implies:

{NV}$^+$ $\rightarrow$ {NV, CRU, DEGRE, MILL} <br>
{NP}$^+$ $\rightarrow$ {NP, NOM, PRENOM, REGION} <br>

To initiate the Boyce-Codd normal form algorithm, use the set {NV} with closure {NV}$^+$ $\rightarrow$ {NV, CRU, DEGRE, MILL}. We have that {NV}$^+$ $\neq$ {NV} and <br>{NV}$^+$ $\neq$ {all attributes}. So we can separate the attributes in:

$R_1$ = {NV, CRU, DEGRE, MILL} <br>
$R_2$ = {NV, QTE, NP, NOM, PRENOM, REGION}

$R_1$ is in Boyce-Codd normal form and is not modified any further. In $R_2$ however the set {NP} has closure { NP}$^+$ = {NOM, PRENOM, REGION}. So we may split $R_2$ into:
$R_{21}$ = {NP, NOM, PRENOM, REGION} 
$R_{22}$ = {NP, NV, QTE}

These two sets are in Boyce-Codd normal form, so we are done.

In the end, we obtain three tables: 

- $R_1$ = "WINE" = {NV, CRU, DEGRE, MILL} with key {NV}
- $R_{21}$ = "SALES" = {NP, NV, QTE} with key {NP,NV}
- $R_{22}$ = "PRODUCER" = {NP, NOM, PRENOM, REGION} with key {NP}

"WINE" relates to "SALES" through NV, and indirectly to "PRODUCER" through the superkey NV, NP.

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

For Master 2, I follow the same logic as Master 1 and and put the table in Boyce-Codd Normal Form. The main functional dependencies for the table are:

NV $\rightarrow$ {NV, CRU, DEGRE, MILL} <br>
NB $\rightarrow$ {NB, NOM, PRENOM, TYPE} <br>
LIEU $\rightarrow$ {LIEU, REGION}

The other dependencies are trivial dependencies, which implies:

NV$^+$ $\rightarrow$ {NV, CRU, DEGRE, MILL} <br>
NB$^+$ $\rightarrow$ {NB, NOM, PRENOM, TYPE} <br>
LIEU$^+$ $\rightarrow$ {LIEU, REGION}

Following, a first step for the Boyce-Codd algorithm consists in starting from the set X = {NV} as NV$^+$ $\rightarrow$ {NV, CRU, DEGRE, MILL} $\neq$ X and NV$^+$ $\neq$ all attributes. This creates two sets:

$R_1$ = {NV, CRU, DEGRE, MILL} <br>
$R_2$ = {NV, DATES, LIEU, QTE, NB, NOM, PRENOM, TYPE, REGION} 

$R_1$ is in Boyce-Codd Normal Form and is not modified further. Using now the set {NB} with NB$^+$ $\rightarrow$ {NB, NOM, PRENOM, TYPE}, one obtains two further sets:

$R_{21}$ = {NB, NOM, PRENOM, TYPE} <br>
$R_{22}$ = {NV, NB, DATES, LIEU, QTE,  REGION}

$R_{21}$ is in Boyce-Codd Normal Form and is not modified further. Finally using the set {LIEU} with LIEU$^+$ $\rightarrow$ {LIEU, REGION}, one obtains a final separation as:

$R_{221}$ = {LIEU, REGION} <br>
$R_{222}$ = {NV, NB, DATES, LIEU, QTE}

This defines a set of four tables:

$R_1$ = "WINE2" = {NV, CRU, DEGRE, MILL} with key NV<br>
$R_{21}$ = "CLIENT" = {NB, NOM, PRENOM, TYPE} with key NB<br>
$R_{221}$ = "LOCATION" = {LIEU, REGION} with key LIEU<br>
$R_{222}$ = "SALES2" = {NV, NB, DATES, LIEU, QTE} with key NV, NB<br>

WINE2 relates to SALES through the attribute NV, and indirectly relates to CLIENT and LOCATION through NB and lieu.

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [7]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [8]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
sales:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
temp3:
  0: NB(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [9]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [10]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Notice that we use `NULL` given that `None` is only used for display.

In [11]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [12]:
%%sql DROP TABLE IF EXISTS wine;

CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [13]:
%%sql DROP TABLE IF EXISTS sales;

CREATE TABLE sales AS
SELECT NP, NV, QTE
FROM MASTER1
WHERE NV IS NOT NULL AND NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [14]:
%%sql DROP TABLE IF EXISTS producer;

CREATE TABLE producer AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [15]:
%%sql DROP TABLE IF EXISTS wine2;

CREATE TABLE wine2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL
ORDER BY NV;

 * sqlite:///wine.db
Done.
Done.


[]

In [16]:
%%sql DROP TABLE IF EXISTS client;

CREATE TABLE client AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL
ORDER BY NB;

 * sqlite:///wine.db
Done.
Done.


[]

In [17]:
%%sql DROP TABLE IF EXISTS location;

CREATE TABLE location AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2
WHERE LIEU IS NOT NULL
ORDER BY LIEU;

 * sqlite:///wine.db
Done.
Done.


[]

In [18]:
%%sql DROP TABLE IF EXISTS sales2;

CREATE TABLE sales2 AS
SELECT distinct NV, NB, DATES, LIEU, QTE
FROM MASTER2
WHERE NV IS NOT NULL AND NB IS NOT NULL
ORDER BY NV;

 * sqlite:///wine.db
Done.
Done.


[]

In [19]:
%%sql
select * from sales2

 * sqlite:///wine.db
Done.


NV,NB,DATES,LIEU,QTE
1,2,1977-11-02,BORDEAUX,33
1,44,2015-10-16,PARIS,1
1,45,1983-12-31,RENNES,1
2,48,1983-12-25,LYON,2
3,7,1978-11-01,NICE,6
3,49,1987-12-24,MERCUREY,5
4,8,1982-11-05,TOULOUSE,12
4,44,2015-10-16,PARIS,6
4,50,1981-06-29,SENS,4
5,2,1984-11-04,MERCUREY,50


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [20]:
%%sql SELECT DISTINCT TYPE
FROM client;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [21]:
%%sql 
SELECT w.CRU, p.REGION
FROM wine w, sales s, producer p
WHERE w.NV = s.NV AND s.NP = p.NP
AND (w.CRU = "Pommard" or w.CRU = "Brouilly");

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [22]:
%%sql 
SELECT p.REGION
FROM wine w, sales s, producer p
WHERE w.NV = s.NV AND s.NP = p.NP
AND (w.CRU = "Pommard")
INTERSECT
SELECT p.REGION
FROM wine w, sales s, producer p
WHERE w.NV = s.NV AND s.NP = p.NP
AND (w.CRU = "Brouilly");

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [23]:
%%sql 
SELECT *
from (SELECT w.CRU, w.MILL, sum(s.QTE) as Number_of_wines_bought
FROM wine2 w, sales2 s
WHERE w.NV = s.NV
group by w.CRU, w.MILL
order by CRU, MILL DESC)
where Number_of_wines_bought IS NOT NULL;

 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1979,520
Beaujolais Villages,1978,130
Beaujolais Villages,1976,120
Beaujolais Villages,1975,10
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [24]:
%%sql 
SELECT NV, count(NP) as Number_of_producers
FROM sales
group by NV
having Number_of_producers > 3;

 * sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [25]:
%%sql 
SELECT producer.NP, producer.NOM, producer.PRENOM
FROM producer LEFT JOIN sales ON (producer.NP = sales.NP)
WHERE NV IS NULL
ORDER BY producer.NP;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [26]:
%%sql
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client c, sales2 s, wine2 w
WHERE c.NB = s.NB AND s.NV = w.NV AND w.MILL = 1980
ORDER BY c.NB;


 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [27]:
%%sql
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client c left join sales2 s on c.NB=s.nb left join wine2 w on s.NV=w.nv
EXCEPT
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client c, sales2 s, wine2 w
WHERE c.NB = s.NB and s.NV = w.NV AND w.MILL = 1980
ORDER BY c.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [29]:
%%sql
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client c, sales2 s, wine2 w
WHERE c.NB = s.NB and s.NV = w.NV AND w.MILL = 1980
EXCEPT
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client c, sales2 s, wine2 w
WHERE c.NB = s.NB and s.NV = w.NV AND w.MILL <> 1980
ORDER BY c.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [30]:
%%sql
SELECT NV, CRU, MILL, DEGRE
FROM wine
WHERE MILL = "1980";

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [31]:
%%sql
SELECT w.NV, w.CRU, w.MILL, w.DEGRE
FROM Wine2 w, sales2 s
WHERE w.MILL = "1980" AND s.NB = 2 AND s.NV = w.NV;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [32]:
%%sql 
-- create table temp1 which contains all clients/wine ID combinations for wines from 1980
DROP TABLE IF EXISTS temp1;
CREATE TABLE temp1 AS
SELECT s.NB, s.NV
FROM sales2 s, wine2 w
where s.NV = w.NV AND NB IS NOT NULL AND MILL = "1980"
ORDER BY NB;



 * sqlite:///wine.db
Done.
Done.


[]

In [33]:
%%sql
-- create table temp2 which contains all wine ID for wines from 1980
DROP TABLE IF EXISTS temp2;
CREATE TABLE temp2 AS
SELECT NV
FROM wine2
where MILL = "1980";

 * sqlite:///wine.db
Done.
Done.


[]

In [34]:
%%sql
-- obtain table temp3 as the relational division of temp1 by temp2
DROP TABLE IF EXISTS temp3;
CREATE TABLE temp3 AS
SELECT DISTINCT t1.NB
FROM temp1 AS t1
WHERE NOT EXISTS (SELECT t2.NV
                FROM temp2 AS t2
                  WHERE NOT EXISTS (SELECT t3.NB
                                   FROM temp1 AS t3
                                   WHERE t3.NB = t1.NB
                                   AND t3.NV = t2.NV)
                );
DROP TABLE IF EXISTS temp1;
DROP TABLE IF EXISTS temp2;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.


[]

In [35]:
%%sql
-- run final query to get first and last names of clients associated with NB
SELECT c.NB, c.NOM, c.PRENOM
FROM client c, temp3 t
WHERE c.NB = t.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
